Load in baseline model and food101

In [ ]:
from datasets import load_dataset

dataset = load_dataset("food101")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})


In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
model_name = "nateraw/food"
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
subset = dataset["train"].shuffle(seed=42).select(range(10000))  # 10k images
print(subset)
import os
import pandas as pd

save_dir = "/content/food101_subset/"
os.makedirs(save_dir, exist_ok=True)

records = []
for idx, row in enumerate(subset):
    img = row["image"]
    label = row["label"]
    filename = f"{idx}.jpg"
    img.save(os.path.join(save_dir, filename))
    records.append({"filename": filename, "label": label})

df_food101 = pd.DataFrame(records)
csv_food101_path = "/content/food101_subset.csv"
df_food101.to_csv(csv_food101_path, index=False)

print("Subset saved to:", save_dir)
print("CSV saved to:", csv_food101_path)
print(df_food101.head())


Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})
Subset saved to: /content/food101_subset/
CSV saved to: /content/food101_subset.csv
  filename  label
0    0.jpg     74
1    1.jpg     71
2    2.jpg     73
3    3.jpg     16
4    4.jpg     63


Load in processed mm-food-100K dataset and do a quick test train split.

In [ ]:
# ============================================================
# 1. Unzip images to /content/
# ============================================================
# This step extracts the dataset into Colab's working directory.

zip_path = "/content/drive/MyDrive/dataset/samples/mmfood_images_224jpg.zip"
extract_dir = "/content/mmfood_images_224"

import os

if not os.path.exists(extract_dir):
    !unzip "{zip_path}" -d "/content/"
else:
    print("Images already extracted.")


# ============================================================
# 2. Load metadata CSV containing image URLs and labels
# ============================================================
# The CSV contains image_url, mapped_or_simplified (label), and other metadata.
# We only extract the filename + label for dataset preparation.

import pandas as pd

csv_path = "/content/drive/MyDrive/dataset/samples/mmfood_for_training.csv"
df = pd.read_csv(csv_path)

print(df.head())
print(df.columns)


# ============================================================
# 3. Extract filename from image_url
# ============================================================
# Example:
# https://file.b18a.io/7843322356500104680_443548_.jpeg
#  → "7843322356500104680_443548_.jpeg"

df["filename"] = df["image_url"].apply(lambda x: os.path.basename(str(x)))


# ============================================================
# 4. Keep only the necessary columns: filename + label
# ============================================================
# We rename the label column to "label" for consistency with PyTorch / HF pipelines.

df_clean = df[["filename", "mapped_or_simplified"]].copy()
df_clean = df_clean.rename(columns={
    "mapped_or_simplified": "label"
})

print(df_clean.head())


Archive:  /content/drive/MyDrive/dataset/samples/mmfood_images_224jpg.zip
replace /content/mmfood_images_224jpg/7815408608800107397_703037_.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
                                           image_url  camera_or_phone_prob  \
0  https://file.b18a.io/7843322356500104680_44354...                   0.7   
1  https://file.b18a.io/7832600581600103585_26423...                   0.8   
2  https://file.b18a.io/7839056601700101188_98515...                   0.7   
3  https://file.b18a.io/7850509358700101451_89218...                   0.8   
4  https://file.b18a.io/7866178489900108040_85120...                   0.7   

   food_prob            dish_name                  food_type  \
0       0.95        Fried Chicken            Restaurant food   
1       0.95  Pan-fried Dumplings            Restaurant food   
2       1.00              Bananas  Raw vegetables and fruits   
3       0.90        Dried Noodles              Packaged food   
4       0.90          Noo

Load in Vireo-172 and do quick test train split.

In [ ]:
# ============================================================
# 1. Unzip Vireo-172 image dataset
# ============================================================
# The images are structured as:  /content/ready_chinese_food/<subfolder>/<filename>
# We extract it to /content/ so we can reference the images easily.

zip_path = "/content/drive/MyDrive/dataset/samples/ready_chinese_food.zip"
extract_dir = "/content/ready_chinese_food"

import os

if not os.path.exists(extract_dir):
    !unzip "{zip_path}" -d "/content/"
else:
    print("Images already extracted.")


# ============================================================
# 2. Load metadata CSV
# ============================================================
import pandas as pd

csv_path = "/content/drive/MyDrive/dataset/samples/vireo_image_mapping_with_foodname.csv"
df = pd.read_csv(csv_path)

print(df.head())
print(df.columns)


# ============================================================
# 3. Extract image path + label
# ============================================================
# We will create a REAL path to the image using:
#    /content/ready_chinese_food/<subfolder_name>/<image_name>
# Also we set "food_name" as the training label.

df["subfolder_name"] = df["subfolder_name"].astype(str)
df["image_name"] = df["image_name"].astype(str)

df["image_path"] = "/content/ready_chinese_food/" + df["subfolder_name"] + "/" + df["image_name"]
df["label"] = df["food_name"]        # rename target column

df_vireo = df[["image_path", "label"]].copy()
df_vireo.head()



Images already extracted.
    image_name  subfolder_name  subfolder_padded     food_name
0        1.jpg               1                 1  Braised Pork
1  1086211.jpg               1                 1  Braised Pork
2     10_0.jpg               1                 1  Braised Pork
3     10_1.jpg               1                 1  Braised Pork
4    10_13.jpg               1                 1  Braised Pork
Index(['image_name', 'subfolder_name', 'subfolder_padded', 'food_name'], dtype='object')


,image_path,label
0,/content/ready_chinese_food/1/1.jpg,Braised Pork
1,/content/ready_chinese_food/1/1086211.jpg,Braised Pork
2,/content/ready_chinese_food/1/10_0.jpg,Braised Pork
3,/content/ready_chinese_food/1/10_1.jpg,Braised Pork
4,/content/ready_chinese_food/1/10_13.jpg,Braised Pork


Combine training datasets - mm-food-100k.

In [ ]:
# ============================================================
# 1. Load mmfood_for_training.csv
# ============================================================
import pandas as pd
import os

csv_path = "/content/drive/MyDrive/dataset/samples/mmfood_for_training.csv"
mmfood_images_root = "/content/mmfood_images_224jpg"  # unzipped folder

df_mm = pd.read_csv(csv_path)
print(df_mm.head())


# ============================================================
# 2. Extract filename from image_url
# ============================================================
df_mm["filename"] = df_mm["image_url"].apply(lambda x: os.path.basename(str(x)))


# ============================================================
# 3. Create full image_path
# ============================================================
df_mm["image_path"] = df_mm["filename"].apply(
    lambda x: os.path.join(mmfood_images_root, x)
)

# Use cleaned label for training
df_mm["label_name"] = df_mm["mapped_or_simplified"].astype(str)

df_mm_clean = df_mm[["image_path", "label_name"]].copy()
print(df_mm_clean.head(), df_mm_clean.shape)


                                           image_url  camera_or_phone_prob  \
0  https://file.b18a.io/7843322356500104680_44354...                   0.7   
1  https://file.b18a.io/7832600581600103585_26423...                   0.8   
2  https://file.b18a.io/7839056601700101188_98515...                   0.7   
3  https://file.b18a.io/7850509358700101451_89218...                   0.8   
4  https://file.b18a.io/7866178489900108040_85120...                   0.7   

   food_prob            dish_name                  food_type  \
0       0.95        Fried Chicken            Restaurant food   
1       0.95  Pan-fried Dumplings            Restaurant food   
2       1.00              Bananas  Raw vegetables and fruits   
3       0.90        Dried Noodles              Packaged food   
4       0.90          Noodle Soup            Restaurant food   

                               ingredients  \
0             ["chicken","breading","oil"]   
1    ["dumplings","chili oil","soy sauce"]   
2       

Combine training datasets - Vireo-172.

In [ ]:
import pandas as pd
import os

# ============================================================
# 1. Load FULL Vireo dataset (not the split!)
# ============================================================
# This CSV contains subfolder + image filename + food label.
vireo_csv = "/content/drive/MyDrive/dataset/samples/vireo_image_mapping_with_foodname.csv"
vireo_root = "/content/ready_chinese_food"  # unzipped folder

df_vireo = pd.read_csv(vireo_csv)
print(df_vireo.head())
print(df_vireo.columns)

# Expected columns:
# image_name | subfolder_name | subfolder_padded | food_name

# ============================================================
# 2. Build absolute image_path
# ============================================================
# Example:
# subfolder=1, image_name=10_1.jpg → /content/ready_chinese_food/1/10_1.jpg
df_vireo["subfolder_name"] = df_vireo["subfolder_name"].astype(str)
df_vireo["image_path"] = df_vireo.apply(
    lambda x: os.path.join(vireo_root, x["subfolder_name"], x["image_name"]),
    axis=1
)

# ============================================================
# 3. Use `food_name` as label
# ============================================================
df_vireo["label_name"] = df_vireo["food_name"].astype(str)

# Keep only final columns
df_vireo_clean = df_vireo[["image_path", "label_name"]].copy()
print(df_vireo_clean.head(), df_vireo_clean.shape)




    image_name  subfolder_name  subfolder_padded     food_name
0        1.jpg               1                 1  Braised Pork
1  1086211.jpg               1                 1  Braised Pork
2     10_0.jpg               1                 1  Braised Pork
3     10_1.jpg               1                 1  Braised Pork
4    10_13.jpg               1                 1  Braised Pork
Index(['image_name', 'subfolder_name', 'subfolder_padded', 'food_name'], dtype='object')
                                  image_path    label_name
0        /content/ready_chinese_food/1/1.jpg  Braised Pork
1  /content/ready_chinese_food/1/1086211.jpg  Braised Pork
2     /content/ready_chinese_food/1/10_0.jpg  Braised Pork
3     /content/ready_chinese_food/1/10_1.jpg  Braised Pork
4    /content/ready_chinese_food/1/10_13.jpg  Braised Pork (110241, 2)


Combine training datasets - Food-101.

In [ ]:
import os
import pandas as pd
from datasets import load_dataset

# Load CSV
df_food101 = pd.read_csv("/content/food101_subset.csv")
print(df_food101.columns)

# STEP 1 — Create full image path
food101_root = "/content/food101_subset"
df_food101["image_path"] = df_food101["filename"].apply(
    lambda x: os.path.join(food101_root, str(x))
)

# STEP 2 — Convert numeric label → clean real-world label
food101 = load_dataset("food101")
int2label = food101["train"].features["label"].int2str

def clean_food101_label(x):
    raw_label = int2label(x)          # e.g. 'chicken_curry'
    cleaned = raw_label.replace('_', ' ').lower().strip()
    return cleaned

df_food101["label_name"] = df_food101["label"].apply(clean_food101_label)

# STEP 3 — Keep only final columns
df_food101_clean = df_food101[["image_path", "label_name"]]
df_food101_clean.to_csv("/content/food101_subset_named.csv", index=False)

print(df_food101_clean.head())
print("Saved to /content/food101_subset_named.csv")


Index(['filename', 'label'], dtype='object')
                      image_path   label_name
0  /content/food101_subset/0.jpg  peking duck
1  /content/food101_subset/1.jpg       paella
2  /content/food101_subset/2.jpg  panna cotta
3  /content/food101_subset/3.jpg   cheesecake
4  /content/food101_subset/4.jpg     macarons
Saved to /content/food101_subset_named.csv


In [ ]:
# ============================================================
# STEP 5: Create a unified label space for all datasets
# ============================================================

# Food101 will act as the base vocabulary.
# All labels from MM-Food and Vireo datasets that are NOT in Food101
# will be added as "new labels".

# 1. Food101 labels (already cleaned earlier)
set_food101 = set(df_food101_clean["label_name"].unique())

# 2. Labels from MM-Food and Vireo-172
mmfood_labels = set(df_mm_clean["label_name"].unique())
vireo_labels = set(df_vireo_clean["label_name"].unique())

# 3. Identify new labels coming from MM-Food / Vireo
#    These are labels that do NOT exist in Food101
new_labels = (mmfood_labels | vireo_labels) - set_food101

# 4. Final label space = Food101 + new labels (union)
final_labels = sorted(list(set_food101 | new_labels))

# 5. Create index mappings
label2id = {lbl: i for i, lbl in enumerate(final_labels)}
id2label = {i: lbl for lbl, i in label2id.items()}

print("Food101 base labels:", len(set_food101))
print("Added MM+Vireo new labels:", len(new_labels))
print("Final label space size:", len(final_labels))

# 6. Save label mapping for future inference consistency
import json
with open("label2id.json", "w", encoding="utf-8") as f:
    json.dump(label2id, f, indent=2, ensure_ascii=False)

with open("id2label.json", "w", encoding="utf-8") as f:
    json.dump(id2label, f, indent=2, ensure_ascii=False)


Food101 base labels: 101
Added MM+Vireo new labels: 561
Final label space size: 662


In [ ]:
# ============================================================
# STEP 6: Convert label_name → label_id using the mapping
# ============================================================

# This ensures all datasets follow a single unified label space.
# If a label is not found in label2id, it means something is wrong.

df_food101_clean["label_id"] = df_food101_clean["label_name"].map(label2id)
df_mm_clean["label_id"] = df_mm_clean["label_name"].map(label2id)
df_vireo_clean["label_id"] = df_vireo_clean["label_name"].map(label2id)

df_all = pd.concat(
    [df_mm_clean, df_vireo_clean, df_food101_clean],
    ignore_index=True
)

# Check for missing mappings
invalid_rows = df_all[df_all["label_id"].isna()]
print("Invalid mappings found:", len(invalid_rows))


Invalid mappings found: 0


/tmp/ipython-input-503785330.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_food101_clean["label_id"] = df_food101_clean["label_name"].map(label2id)


In [ ]:
# ============================================================
# STEP 7: Merge all datasets into a single dataframe
# ============================================================

df_all = pd.concat([
    df_food101_clean[["image_path", "label_name", "label_id"]],
    df_mm_clean[["image_path", "label_name", "label_id"]],
    df_vireo_clean[["image_path", "label_name", "label_id"]]
], ignore_index=True)

print("Total dataset size:", df_all.shape)
print("Label space:", len(label2id))
print(df_all.head())


Total dataset size: (187895, 3)
Label space: 662
                      image_path   label_name  label_id
0  /content/food101_subset/0.jpg  peking duck       487
1  /content/food101_subset/1.jpg       paella       477
2  /content/food101_subset/2.jpg  panna cotta       480
3  /content/food101_subset/3.jpg   cheesecake       247
4  /content/food101_subset/4.jpg     macarons       424


In [ ]:
# ============================================================
# STEP 8: Convert pandas dataframe → HuggingFace Dataset
# ============================================================

from datasets import Dataset

hf_ds = Dataset.from_pandas(df_all)

# Sample one row to verify structure
print(hf_ds[0])


{'image_path': '/content/food101_subset/0.jpg', 'label_name': 'peking duck', 'label_id': 487}


Model training

In [ ]:
# 80 / 20 split
split = hf_ds.train_test_split(test_size=0.2, seed=42)
train_ds = split["train"]
val_ds = split["test"]

print(len(train_ds), len(val_ds))


150316 37579


In [ ]:
def map_label(example):
    example["label_id"] = label2id[example["label_name"]]   # harmonized_name
    return example

train_ds = train_ds.map(map_label)
val_ds   = val_ds.map(map_label)


Map:   0%|          | 0/150316 [00:00<?, ? examples/s]

Map:   0%|          | 0/37579 [00:00<?, ? examples/s]

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class HFImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        path = example["image_path"]
        label = int(example["label_id"])

        img = Image.open(path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return {"pixel_values": img, "labels": label}


In [ ]:
missing = []

for i in range(len(train_ds)):
    path = train_ds[i]["image_path"]
    if not os.path.exists(path):
        missing.append(path)

len(missing)



9673

In [ ]:
def exists(example):
    return os.path.exists(example["image_path"])

train_ds = train_ds.filter(exists)
val_ds   = val_ds.filter(exists)


Filter:   0%|          | 0/150316 [00:00<?, ? examples/s]

Filter:   0%|          | 0/37579 [00:00<?, ? examples/s]

In [ ]:
print(len(train_ds), len(val_ds))
# test again
print(os.path.exists(train_ds[0]["image_path"]))


140643 35207
True


In [ ]:
train_dataset = HFImageDataset(train_ds, transform=transform)
val_dataset   = HFImageDataset(val_ds,   transform=transform)


In [ ]:
labels = list(train_ds["label_id"])

print("Min label:", min(labels))
print("Max label:", max(labels))
print("Number of classes:", len(set(labels)))
print("Example labels:", sorted(list(set(labels))[:10]))



Min label: 0
Max label: 661
Number of classes: 662
Example labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
from transformers import AutoModelForImageClassification

num_classes = len(label2id)  # 662

model = AutoModelForImageClassification.from_pretrained(
    "nateraw/food",
    num_labels=num_classes,
    ignore_mismatched_sizes=True,
)

print(model.config.num_labels)  # 必须输出 662！


Some weights of ViTForImageClassification were not initialized from the model checkpoint at nateraw/food and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([101, 768]) in the checkpoint and torch.Size([662, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([101]) in the checkpoint and torch.Size([662]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


662


In [ ]:
print(model.config.num_labels)


662


In [ ]:
print(model)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
from PIL import Image
from dataclasses import dataclass
import numpy as np

# 1. load processor
processor = AutoImageProcessor.from_pretrained("nateraw/food")

# 2. define Dataset
class LazyImageDataset(Dataset):
    def __init__(self, hf_dataset, processor):
        self.dataset = hf_dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        image = Image.open(example["image_path"]).convert("RGB")
        inputs = self.processor(image, return_tensors="pt")

        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "labels": torch.tensor(example["label_id"], dtype=torch.long)
        }

# 3. define Collator
@dataclass
class ImageClassificationCollator:
    def __call__(self, features):
        batch = {
            "pixel_values": torch.stack([f["pixel_values"] for f in features]),
            "labels": torch.stack([f["labels"] for f in features])
        }
        return batch

# 4. create datasets
train_dataset = LazyImageDataset(train_ds, processor)
val_dataset = LazyImageDataset(val_ds, processor)

# 5. load model
model = AutoModelForImageClassification.from_pretrained(
    "nateraw/food",
    num_labels=662,
    ignore_mismatched_sizes=True,
)

# 6. evaluate
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # accuracy
    accuracy = (predictions == labels).mean()

    # Top-5 accuracy
    top5_predictions = np.argsort(eval_pred[0], axis=1)[:, -5:]
    top5_accuracy = np.mean([labels[i] in top5_predictions[i] for i in range(len(labels))])

    return {
        "accuracy": accuracy,
        "top5_accuracy": top5_accuracy
    }

# 7. train parameters
args = TrainingArguments(
    output_dir="./food-classifier-output",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    bf16=True,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    remove_unused_columns=False,
)

# 8. create Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=ImageClassificationCollator(),
    compute_metrics=compute_metrics,
)

# 9. start trainning
trainer.train()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at nateraw/food and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([101, 768]) in the checkpoint and torch.Size([662, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([101]) in the checkpoint and torch.Size([662]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,Top5 Accuracy
500,5.718600,5.674112,0.092340,0.291022
1000,5.076100,5.083273,0.250774,0.515693
1500,4.594600,4.573081,0.332036,0.617349
2000,4.139300,4.124670,0.400829,0.676314
2500,3.824500,3.740117,0.456244,0.720027
3000,3.468900,3.414407,0.497373,0.743659
3500,3.141000,3.127199,0.529099,0.766836
4000,2.918600,2.892709,0.557474,0.778283
4500,2.555700,2.677940,0.585736,0.794103
5000,2.377500,2.499819,0.603772,0.807055


TrainOutput(global_step=13188, training_loss=2.560499693720421, metrics={'train_runtime': 4641.5937, 'train_samples_per_second': 90.902, 'train_steps_per_second': 2.841, 'total_flos': 3.288952736032507e+19, 'train_loss': 2.560499693720421, 'epoch': 3.0})

In [ ]:
from google.colab import drive


!mkdir -p /content/drive/MyDrive/food_classifier_model

# 3. save final model
print("Saving final model to Google Drive...")
trainer.save_model("/content/drive/MyDrive/food_classifier_model/final_model")

# 4. save label mapping
import pickle

with open('/content/drive/MyDrive/food_classifier_model/label2id.pkl', 'wb') as f:
    pickle.dump(label2id, f)

with open('/content/drive/MyDrive/food_classifier_model/id2label.pkl', 'wb') as f:
    pickle.dump(id2label, f)

# 5. save processor
processor.save_pretrained("/content/drive/MyDrive/food_classifier_model/processor")

# 6. save
import json

training_args_dict = args.to_dict()
with open('/content/drive/MyDrive/food_classifier_model/training_args.json', 'w') as f:
    json.dump(training_args_dict, f, indent=2)

print("✅ All files saved to Google Drive!")
print("Location: /content/drive/MyDrive/food_classifier_model/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving final model to Google Drive...
✅ All files saved to Google Drive!
Location: /content/drive/MyDrive/food_classifier_model/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import torch

def detailed_evaluation(model, dataset, processor, id2label):
    """详细评估模型性能"""

    model.eval()
    device = model.device

    all_preds = []
    all_labels = []
    all_probs = []

    print("Running inference on validation set...")

    # gothrough all dataset
    for i in range(len(dataset)):
        example = dataset.dataset[i]
        image = Image.open(example["image_path"]).convert("RGB")
        true_label = example["label_id"]

        # predict
        inputs = processor(image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=-1)[0]
            pred = outputs.logits.argmax(-1).item()

        all_preds.append(pred)
        all_labels.append(true_label)
        all_probs.append(probs.cpu().numpy())

        if (i + 1) % 1000 == 0:
            print(f"  Processed {i + 1}/{len(dataset)} samples")

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # 1. overall accuracy
    accuracy = (all_preds == all_labels).mean()

    # 2. Top-5 accuracy
    top5_preds = np.argsort(all_probs, axis=1)[:, -5:]
    top5_acc = np.mean([all_labels[i] in top5_preds[i] for i in range(len(all_labels))])

    # 3. accuracy
    per_class_acc = {}
    for label_id in range(662):
        mask = all_labels == label_id
        if mask.sum() > 0:
            per_class_acc[label_id] = (all_preds[mask] == label_id).mean()

    # 4. best class vs worst class
    sorted_classes = sorted(per_class_acc.items(), key=lambda x: x[1], reverse=True)

    print("\n" + "=" * 60)
    print("📊 DETAILED EVALUATION RESULTS")
    print("=" * 60)
    print(f"Overall Accuracy:     {accuracy:.2%}")
    print(f"Top-5 Accuracy:       {top5_acc:.2%}")
    print(f"\n🏆 Top 10 Best Performing Classes:")
    for label_id, acc in sorted_classes[:10]:
        print(f"  {id2label[label_id]:30s} {acc:.2%}")

    print(f"\n😞 Top 10 Worst Performing Classes:")
    for label_id, acc in sorted_classes[-10:]:
        print(f"  {id2label[label_id]:30s} {acc:.2%}")

    return {
        'accuracy': accuracy,
        'top5_accuracy': top5_acc,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs,
        'per_class_accuracy': per_class_acc
    }

# eval results
eval_results = detailed_evaluation(model, val_dataset, processor, id2label)

Running inference on validation set...
  Processed 1000/35207 samples
  Processed 2000/35207 samples
  Processed 3000/35207 samples
  Processed 4000/35207 samples
  Processed 5000/35207 samples
  Processed 6000/35207 samples
  Processed 7000/35207 samples
  Processed 8000/35207 samples
  Processed 9000/35207 samples
  Processed 10000/35207 samples
  Processed 11000/35207 samples
  Processed 12000/35207 samples
  Processed 13000/35207 samples
  Processed 14000/35207 samples
  Processed 15000/35207 samples
  Processed 16000/35207 samples
  Processed 17000/35207 samples
  Processed 18000/35207 samples
  Processed 19000/35207 samples
  Processed 20000/35207 samples
  Processed 21000/35207 samples
  Processed 22000/35207 samples
  Processed 23000/35207 samples
  Processed 24000/35207 samples
  Processed 25000/35207 samples
  Processed 26000/35207 samples
  Processed 27000/35207 samples
  Processed 28000/35207 samples
  Processed 29000/35207 samples
  Processed 30000/35207 samples
  Processe